In [2]:
import cv2
import mediapipe as mp


In [3]:
mp_hand = mp.solutions.hands
hands = mp_hand.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
drawing_styles = mp_drawing_styles.DrawingSpec(thickness=2, circle_radius=2)


can = cv2.VideoCapture(0)
while can.isOpened():
    ret, frame = can.read()
    if not ret:
        print("i'm gonna ignore empty camera frame")
        continue
    # Convert the BGR image to RGB and process it with MediaPipe Hands.
    frame = cv2.cvtColor(cv2.flip(frame,1), cv2.COLOR_BGR2RGB)
    # Prevent writing to improve performance.
    frame.flags.writeable = False

    #processing the img
    results = hands.process(frame)
    
    count=0

    #converting to the original form
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    frame.flags.writeable = True

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:

            #getting the positions of landmarks
            landmarks = hand_landmarks.landmark

            #making a list of fingertips pos
            tips = [4,8,12,16,20]
            #another list for one position down of the tips
            belowtips = [3,7,11,15,19]

            for i in range(5):
                #thumb
                if i==0:
                    if landmarks[tips[i]].x < landmarks[belowtips[i]].x:
                        count+=1
                #other fingers
                else:
                    if landmarks[tips[i]].y < landmarks[belowtips[i]].y:
                        count+=1
            
            cv2.putText(frame, str(count), (10,70), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)
            mp_draw.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hand.HAND_CONNECTIONS,
                drawing_styles,
                drawing_styles
            )
    cv2.imshow('Finger Counting', frame)

    if cv2.waitKey(5) & 0xFF == 27:  #Press 'Esc' to exit
        break

can.release()
cv2.destroyAllWindows()